In [ ]:
#!g1.1
!git clone https://github.com/nerel-ds/NEREL

In [ ]:
!unzip NEREL-v1.0.zip -d NEREL

In [1]:
#!g1.1
## Read the data# Reading files
from collections import namedtuple
import re
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import nltk

Ann = namedtuple('annotation', ['tag', 'start1', 'end1', 'start2', 'end2', 'text'])
Rel = namedtuple('relationship', ['tag', 'arg1', 'arg2'])

def read_files(folder):
    filenames = sorted(set(e[:e.rfind('.')] for e in os.listdir(folder)))[1:]
    
    texts, entities, relationships  = [], [], []
    for file in tqdm(filenames):
       # if (not file): continue
        path1 = os.path.join(folder, file+'.txt')
        path2 = os.path.join(folder, file+'.ann')
        if (not os.path.exists(path2)):
            print(f'{path2} not found')
            continue
            #with open(path2, 'w'):
            #    pass
            
        with open(path1, 'r', encoding="utf8") as text, open(path2, 'r', encoding="utf8") as ann:
            texts.append(text.read())

            file_entities = {}
            file_relationship = []
            regex_ent = r'T(?P<id>\d+)\s(?P<tag>\w+)\s(?P<start1>\d+) (?P<end1>\d+)(;(?P<start2>\d+) (?P<end2>\d+))?\s(?P<text>.*)'
            regex_rel = r'R(?P<id>\d+)\s(?P<tag>\w+)\sArg1:T(?P<arg1>\d+) Arg2:T(?P<arg2>\d+)'
            
            
            for row in sorted(ann.readlines(), reverse=True):
                #print(row)
                match_ent = re.match(regex_ent, row)
                match_rel = re.match(regex_rel, row)
                if (match_ent):
                    res = match_ent.groupdict()
                    res['start1'] = int(res['start1'])
                    res['end1'] = int(res['end1'])
                    if (res['start2'] is not None):
                        res['start2'] = int(res['start2'])
                        res['end2'] = int(res['end2'])
                    id = res.pop('id')
                    file_entities[id] = Ann(**res)
                elif (match_rel):
                    try:
                        res = match_rel.groupdict()
                        res['arg1'] = file_entities[res['arg1']]
                        res['arg2'] = file_entities[res['arg2']]
                        id = res.pop('id')
                        file_relationship.append(Rel(**res))
                    except KeyError as e:
                        print(f'not found T{e} row={row}')
                else:
                    print(f'incorrect format in: row={row} file={file}')
            entities.append(file_entities)
            relationships.append(file_relationship)
    entities = [sorted(e.values(), key = lambda x: (x.start1, x.end1)) for e in entities]
    return texts, entities, relationships, filenames

In [2]:
#!g1.1
folder = 'NEREL/NEREL-v1.0/train'
texts, entities, relationships, filenames = read_files(folder)

incorrect format in: row=По словам очевидцев пешехо
 file=21013_text


100%|██████████| 745/745 [00:01<00:00, 450.44it/s]


In [3]:
#!g1.1
def in_range(range1, range2):
    # range1 in range2
    if range1[0] is None: return True
    if range2[0] is None: return False
    return int(range2[0]) <= int(range1[0]) and int(range2[1]) >= int(range1[1])

def is_nested_anns(ent1: Ann, ent2: Ann):
    # ent 1 in ent 2
    res = True
    ent1_range1 = (ent1.start1, ent1.end1)
    ent1_range2 = (ent1.start2, ent1.end2)
    ent2_range1 = (ent2.start1, ent2.end1)
    ent2_range2 = (ent2.start2, ent2.end2)
    #print(f'{ent1_range1=} {ent1_range2=} {ent1_range2=} {ent2_range2=}')
    res = res and (in_range(ent1_range1, ent2_range1) or in_range(ent1_range1, ent2_range2))
    res = res and (in_range(ent1_range2, ent2_range1) or in_range(ent1_range2, ent2_range2))
    return res


def is_nested_anns2(ent1: Ann, ent2: Ann):
    return is_nested_anns(ent1,ent2) or is_nested_anns(ent2, ent1)

def is_nested(rel: Rel):
    return is_nested_anns(rel.arg1, rel.arg2) or is_nested_anns(rel.arg2, rel.arg1)

![](https://i.imgur.com/tgDfc8i.png)             | ![](https://i.imgur.com/oWa5vWo.png)
:-------------------------:|:-------------------------:


In [4]:
#!g1.1
%pip install transformers

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


# Dataset

In [5]:
#!g1.1
import os
import torch
import time
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
from tqdm.notebook import tqdm
import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from matplotlib import pyplot as plt
import random
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [6]:
#!g1.1
from transformers import BertForSequenceClassification, BertTokenizer

In [7]:
#!g1.1
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


In [8]:
#!g1.1
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased-sentence')

In [9]:
#!g1.1
MAX_LENGTH = 100

In [10]:
#!g1.1
# import nltk
# nltk.download('punkt')
# #!g1.1
# sents = nltk.sent_tokenize(texts[0])
# len(' '.join(sents))

In [11]:
#!g1.1
class Binary_Tag:
    def __getitem__(self, x):
        if (x == 'None'): return 0
        else: return 1

# tags = list(set(e.tag for e in sum(relationships,[]))) + ['None'] # ids --> string tag
reversed_tags = Binary_Tag() # string tag --> ids
entity_tags = list(set(e.tag for e in sum(entities,[]))) 

def tok(text1, text2):
    # longest entity is 36 tokens long
    res = tokenizer.encode_plus(text1, text2,
                      max_length = MAX_LENGTH,
                      pad_to_max_length = True,
                      return_tensors = 'pt',
                      )
    return res['input_ids'][0], res['attention_mask'][0], res['token_type_ids'][0]

def order(e1, e2):
    if (len(e1.text) > len(e2.text)):
        return e1, e2
    else:
        return e2, e1


class MyDataset(torch.utils.data.Dataset):
    def __init__(self, folder):
        texts, entities, relationships, filenames = read_files(folder)
        
        pre_dataset = []

        for text_id in range(len(texts)):
            relationships_nested = {(e.arg1, e.arg2):e for e in relationships[text_id] if is_nested(e)}
            nes = []
            for i in range(len(entities[text_id])):
                # O(n^2) eeeeeeeeee
                for j in range(i+1, len(entities[text_id])):
                    if (is_nested_anns2(entities[text_id][i], entities[text_id][j])):
                        nes.append((entities[text_id][i], entities[text_id][j]))
            for e in nes:
                if (e in relationships_nested):
                    pre_dataset.append((*order(*e), relationships_nested[e].tag, text_id))
                elif ((e[1], e[0]) in relationships_nested):
                    pre_dataset.append((*order(*e), relationships_nested[(e[1], e[0])].tag, text_id))
                else:
                    pre_dataset.append((*order(*e), 'None', text_id))

        data = []
        for e1, e2, tag, text_id in tqdm(pre_dataset):
            
            data.append((reversed_tags[tag], entity_tags.index(e1.tag), entity_tags.index(e2.tag), tag, *tok(e1.text, e2.text)))
        
        self.data = data
    
    def __getitem__(self, index):
        return self.data[index]
    
    def __len__(self):
        return len(self.data)
    

In [12]:
#!g1.1
def findPair(i, all_entities, all_relationships_dict):
    j = (i+1)%len(all_entities)
    while True:
        ent1 = all_entities[i]
        ent2 = all_entities[j]
        j = (j+1)%len(all_entities)
        if ((ent1, ent2) not in all_relationships_dict and (ent2, ent1) not in all_relationships_dict):
            return ent1, ent2


class MyDataset2(torch.utils.data.Dataset):
    def __init__(self, folder):
        texts, entities, relationships, filenames = read_files(folder)
        
        random.seed(2021)
        data = []
        
        for r in tqdm(sum(relationships,[]), desc = 'real tags'):
            e1, e2 = order(r.arg1, r.arg2)
            data.append((reversed_tags[r.tag],  entity_tags.index(e1.tag), entity_tags.index(e2.tag), r.tag, *tok(e1.text, e2.text)))
        
        all_entities = sum(entities, [])
        all_relationships_dict = {(e.arg1, e.arg2):e for e in sum(relationships,[])}
        for _ in tqdm(range(len(sum(relationships,[]))), desc = "'None' tags"):
#             e1, e2 = random.sample(all_entities, 2)
            ent1_id = random.choice(range(len(all_entities)))
            e1, e2 = order(*findPair(ent1_id, all_entities, all_relationships_dict))
            data.append((reversed_tags['None'],  entity_tags.index(e1.tag), entity_tags.index(e2.tag), 'None', *tok(e1.text, e2.text)))

        self.data = data
    
    def __getitem__(self, index):
        return self.data[index]
    
    def __len__(self):
        return len(self.data)

In [23]:
#!g1.1
train_data_mixed = MyDataset2('NEREL/NEREL-v1.0/train')
train_dl_mixed = DataLoader(train_data_mixed, shuffle=True, batch_size=32)

train_data_normal = MyDataset('NEREL/NEREL-v1.0/train')
train_dl_normal = DataLoader(train_data_normal, shuffle=True, batch_size=32)

dev_data_mixed = MyDataset2('NEREL/NEREL-v1.0/dev')
dev_dl_mixed = DataLoader(dev_data_mixed, batch_size=32)

dev_data_normal = MyDataset('NEREL/NEREL-v1.0/dev')
dev_dl_normal = DataLoader(dev_data_normal, batch_size=32)

incorrect format in: row=По словам очевидцев пешехо
 file=21013_text



incorrect format in: row=По словам очевидцев пешехо
 file=21013_text









# Training

In [16]:
#!g1.1
from transformers import BertTokenizer, BertModel
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert = BertModel.from_pretrained(
            "DeepPavlov/rubert-base-cased-sentence", # Use the 12-layer BERT model, with an uncased vocab.
            output_attentions = False, # Whether the model returns attentions weights.
            output_hidden_states = False, # Whether the model returns all hidden-states.
            max_length = MAX_LENGTH
            )
        self.dropout = nn.Dropout(0.1)
        self.emb = nn.Embedding(29, 10)
        self.fc = nn.Sequential(
            nn.Linear(768 + 10*2, 200),
            nn.LeakyReLU(True),
            nn.Linear(200, 200),
            nn.LeakyReLU(True),
            nn.Linear(200, 2)
            )
        
    def forward(self, ent1, ent2, input_ids, attention_mask, token_type_ids, labels):
        ent1 = self.emb(ent1)
        ent2 = self.emb(ent2)
        out = self.bert(input_ids = input_ids, token_type_ids = token_type_ids, attention_mask = attention_mask)[1]
        out = self.dropout(out)
        out = torch.cat((out, ent1, ent2), dim=1)
        out = self.fc(out)
        criterion = nn.CrossEntropyLoss()
        loss = criterion(out.view(-1, 2), labels.view(-1))
        return loss, out
    
        # requires reconfiguration of hyper parameters
#         criterion = nn.BCEWithLogitsLoss()
#         out = self.fc(out).view(-1)
#         loss = criterion(out, labels)
#         return loss, torch.stack((-torch.sigmoid(out), torch.sigmoid(out)), dim = 1)

In [24]:
#!g1.1
model = MyModel()
model.to(device)

MyModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
   

In [25]:
#!g1.1
def calculate_accuracy(y_pred, y):
    return sum(torch.argmax(y_pred, axis = 1) == y)*1.0/len(y)

def train(train_dl):
    model.train()
    running_loss = 0
    epoch_accuracy = 0

    pbar = tqdm(enumerate(train_dl), total = len(train_dl))
    for i, (labels, tag1, tag2, true_tag, input_ids, attention_mask, token_type_ids) in pbar:
        
        labels = labels.to(device)
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        tag1 = tag1.to(device)
        tag2 = tag2.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(tag1, tag2,
                    input_ids, 
                    token_type_ids=token_type_ids,
                    attention_mask=attention_mask, 
                    labels=labels)
        
        loss = outputs[0]
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        epoch_accuracy += calculate_accuracy(outputs[1], labels).item()
        running_loss += loss.item()
        pbar.set_description(f'training: running_loss = {running_loss/(i+1.0):.4f} accuracy = {epoch_accuracy/(i+1.0):.4f},')
    
    scheduler.step(running_loss/(i+1))
    print(f'train loss= {running_loss/(i+1):.4f} \n train accuracy = {epoch_accuracy/(i+1):.4f},')

    
from sklearn.metrics import f1_score
def test(dev_dl):
    with torch.no_grad():
        model.eval()
        running_loss = 0
        epoch_accuracy = 0

#         dev_dl = DataLoader( MyDataset('NEREL/NEREL-v1.0/train'), batch_size = 32)

        pbar = tqdm(enumerate(dev_dl), total = len(dev_dl))

        y_true = []
        y_pred = []

        for i, (labels, tag1, tag2, true_tag, input_ids, attention_mask, token_type_ids) in pbar:
            labels = labels.to(device)
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            tag1 = tag1.to(device)
            tag2 = tag2.to(device)

            outputs = model(tag1, tag2,
                        input_ids, 
                        token_type_ids=token_type_ids,
                        attention_mask=attention_mask, 
                        labels=labels)

            loss = outputs[0]
#             calculate_accuracy(outputs[1], labels)
            y_true += [e.item() for e in labels]
            y_pred += [e.item() for e in torch.argmax(outputs[1], axis = 1)]
            
            running_loss += loss.item()
            epoch_accuracy += calculate_accuracy(outputs[1], labels).item()
            # pbar.set_description(f'testing: running_loss = {running_loss/(i+1):.4f} accuracy = {epoch_accuracy/(i+1):.4f},')
#     print(y_true, y_pred)
    print(f'test loss= {running_loss/(i+1):.4f} \n test accuracy = {epoch_accuracy/(i+1):.4f}, F1 = {f1_score(y_true, y_pred)}')

In [26]:
#!g1.1
optimizer = torch.optim.AdamW(model.parameters(),
                  lr = 5e-5, # args.learning_rate 
                  eps = 5e-8#1e-8 # args.adam_epsilon 
                )
scheduler = ReduceLROnPlateau(optimizer, patience = 3, cooldown = 1, factor = 0.5)

In [27]:
#!g1.1
seed_val = 2021

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
for epoch in range(1, 6):
    print(f'epoch = {epoch}')
    print('lr=', optimizer.param_groups[0]['lr'])
    train(train_dl_mixed)
    print('mixed dataset')
    test(dev_dl_mixed)
    print('nested dataset')
    test(dev_dl_normal)

epoch = 1
lr= 5e-05

train loss= 0.4716 
 train accuracy = 0.7776,
mixed dataset

test loss= 0.4970 
 test accuracy = 0.7671, F1 = 0.7588605055503544
nested dataset

test loss= 0.4004 
 test accuracy = 0.8345, F1 = 0.7950428979980935
epoch = 2
lr= 5e-05

train loss= 0.3339 
 train accuracy = 0.8635,
mixed dataset

test loss= 0.5083 
 test accuracy = 0.7778, F1 = 0.7894736842105262
nested dataset

test loss= 0.4354 
 test accuracy = 0.8334, F1 = 0.8011049723756906
epoch = 3
lr= 5e-05

train loss= 0.2353 
 train accuracy = 0.9108,
mixed dataset

test loss= 0.5789 
 test accuracy = 0.7780, F1 = 0.7863184079601989
nested dataset

test loss= 0.5473 
 test accuracy = 0.8040, F1 = 0.77088948787062
epoch = 4
lr= 5e-05

train loss= 0.1819 
 train accuracy = 0.9367,
mixed dataset

test loss= 0.7588 
 test accuracy = 0.7626, F1 = 0.7571994715984148
nested dataset

test loss= 0.6085 
 test accuracy = 0.8280, F1 = 0.7781094527363184
epoch = 5
lr= 5e-05

train loss= 0.1510 
 train accuracy = 0.9501,

In [29]:
#!g1.1
for epoch in range(1, 11):
    print(f'epoch = {epoch}')
    print('lr=', optimizer.param_groups[0]['lr'])
    train(train_dl_normal)
    print('nested dataset')
    test(dev_dl2_normal)

epoch = 1
lr= 5e-05

train loss= 0.2412 
 train accuracy = 0.9104,
nested dataset

test loss= 0.3742 
 test accuracy = 0.8604, F1 = 0.8213228035538006
epoch = 2
lr= 5e-05

train loss= 0.1632 
 train accuracy = 0.9447,
nested dataset

test loss= 0.4854 
 test accuracy = 0.8570, F1 = 0.8121212121212121
epoch = 3
lr= 5e-05

train loss= 0.1332 
 train accuracy = 0.9573,
nested dataset

test loss= 0.5538 
 test accuracy = 0.8555, F1 = 0.8081632653061225
epoch = 4
lr= 5e-05

train loss= 0.1142 
 train accuracy = 0.9633,
nested dataset

test loss= 0.4875 
 test accuracy = 0.8685, F1 = 0.8311944718657452
epoch = 5
lr= 5e-05

train loss= 0.1013 
 train accuracy = 0.9678,
nested dataset

test loss= 0.6060 
 test accuracy = 0.8574, F1 = 0.8194174757281554
epoch = 6
lr= 5e-05

train loss= 0.0839 
 train accuracy = 0.9712,
nested dataset

test loss= 0.6231 
 test accuracy = 0.8407, F1 = 0.7828810020876826
epoch = 7
lr= 5e-05

train loss= 0.0863 
 train accuracy = 0.9724,
nested dataset

test loss= 

In [30]:
#!g1.1
for epoch in range(1, 11):
    print(f'epoch = {epoch}')
    print('lr=', optimizer.param_groups[0]['lr'])
    train(train_dl_normal)
    print('nested dataset')
    test(dev_dl2_normal)

epoch = 1
lr= 5e-05

train loss= 0.0717 
 train accuracy = 0.9790,
nested dataset

test loss= 0.7079 
 test accuracy = 0.8532, F1 = 0.804503582395087
epoch = 2
lr= 5e-05

train loss= 0.0784 
 train accuracy = 0.9783,
nested dataset

test loss= 0.5790 
 test accuracy = 0.8502, F1 = 0.8028311425682508
epoch = 3
lr= 5e-05

train loss= 0.0654 
 train accuracy = 0.9814,
nested dataset

test loss= 0.7150 
 test accuracy = 0.8631, F1 = 0.8209255533199196
epoch = 4
lr= 5e-05

train loss= 0.0614 
 train accuracy = 0.9794,
nested dataset

test loss= 0.7968 
 test accuracy = 0.8525, F1 = 0.8048780487804879
epoch = 5
lr= 5e-05

train loss= 0.0653 
 train accuracy = 0.9793,
nested dataset

test loss= 0.7297 
 test accuracy = 0.8551, F1 = 0.803347280334728
epoch = 6
lr= 5e-05

train loss= 0.0601 
 train accuracy = 0.9824,
nested dataset

test loss= 0.6665 
 test accuracy = 0.8608, F1 = 0.8169868554095047
epoch = 7
lr= 5e-05

train loss= 0.0606 
 train accuracy = 0.9817,
nested dataset

test loss= 0.

In [432]:
#!g1.1
torch.save(model.state_dict(), 'models/Set2_BERT_Binary_tags_mixed_model2')

In [470]:
#!g1.1
with torch.no_grad():
    model.eval()
    running_loss = 0
    epoch_accuracy = 0

    dev_dl = DataLoader( MyDataset('NEREL/NEREL-v1.0/dev'), batch_size = 1)

    pbar = tqdm(enumerate(dev_dl), total = len(dev_dl))

    y_true = []
    y_pred = []
    tags = []
    
    mis_tags = []
    rel_tags = []
    texts = []
    found_expected = []

    for i, (labels, tag1, tag2, true_tag, input_ids, attention_mask, token_type_ids) in pbar:
        labels = labels.to(device)
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        tag1 = tag1.to(device)
        tag2 = tag2.to(device)

        outputs = model(tag1, tag2,
                        input_ids, 
                        token_type_ids=token_type_ids,
                        attention_mask=attention_mask, 
                        labels=labels)

        loss = outputs[0]
#             calculate_accuracy(outputs[1], labels)
        y_true += [e.item() for e in labels]
        y_pred += [e.item() for e in torch.argmax(outputs[1], axis = 1)]
        tags += [(entity_tags[tag1], entity_tags[tag2])]
        
        running_loss += loss.item()
        epoch_accuracy += calculate_accuracy(outputs[1], labels).item()
        if (torch.argmax(outputs[1], axis = 1) != labels):
            print('expected:', labels.detach().cpu().item())
            print('found:', torch.argmax(outputs[1], axis = 1).detach().cpu().item(), '(', outputs[1].detach().cpu().numpy() ,')')
            print(f'{tags[-1][0]} ---> {tags[-1][0]} : {true_tag[0]}')
            print(tokenizer.decode(input_ids[0].detach().cpu().numpy()).replace(r'[PAD]', ''))
            
            
            mis_tags.append(tags[-1])
            texts.append(tokenizer.decode(input_ids[0].detach().cpu().numpy()).replace(r'[PAD]', ''))
            found_expected.append((torch.argmax(outputs[1], axis = 1).detach().cpu().item(), labels.detach().cpu().item()))
            rel_tags.append(true_tag[0])
            print()
            
print(f'test loss= {running_loss/(i+1):.4f} \n test accuracy = {epoch_accuracy/(i+1):.4f}, F1 = {f1_score(y_true, y_pred)}')



expected: 0
found: 1 ( [[-2.4469059  2.9365847]] )
ORGANIZATION ---> ORGANIZATION : None
[CLS] Строгановское высшее художественное училище [SEP] Строгановское [SEP]                                                                                          

expected: 0
found: 1 ( [[-2.7719362  3.3657992]] )
EVENT ---> EVENT : None
[CLS] Международного музыкального фестиваля « Ереванские перспективы [SEP] Ереванские [SEP]                                                                                        

expected: 1
found: 0 ( [[ 3.0674367 -3.5058205]] )
EVENT ---> EVENT : TAKES_PLACE_IN
[CLS] Ереванские перспективы [SEP] Ереванские [SEP]                                                                                            

expected: 1
found: 0 ( [[ 3.139362  -3.6652758]] )
PROFESSION ---> PROFESSION : ORIGINS_FROM
[CLS] сотрудников исправительных учреждений Италии [SEP] Италии [SEP]                                                                                            



/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:859: UserWarning: The following variables cannot be serialized: pbar
  warnings.warn(message)


In [446]:
#!g1.1
sum([a != b for a,b in zip(y_true, y_pred)])

194

In [448]:
#!g1.1
len(mis_tags)

194

In [463]:
#!g1.1
print('\n'.join([f'{a} ---> {b} : {n}' for (a,b),n in Counter(mis_tags).most_common()]))

PROFESSION ---> ORGANIZATION : 25
ORGANIZATION ---> COUNTRY : 14
EVENT ---> LOCATION : 12
EVENT ---> PERSON : 10
ORGANIZATION ---> ORGANIZATION : 9
PROFESSION ---> COUNTRY : 8
ORGANIZATION ---> STATE_OR_PROVINCE : 8
ORGANIZATION ---> PERSON : 6
EVENT ---> COUNTRY : 5
PROFESSION ---> PROFESSION : 5
PROFESSION ---> STATE_OR_PROVINCE : 5
FACILITY ---> PERSON : 5
ORGANIZATION ---> RELIGION : 4
ORGANIZATION ---> PROFESSION : 4
EVENT ---> EVENT : 4
WORK_OF_ART ---> PERSON : 4
ORGANIZATION ---> CITY : 4
ORGANIZATION ---> IDEOLOGY : 4
EVENT ---> CITY : 3
ORGANIZATION ---> LOCATION : 3
EVENT ---> DISEASE : 3
LAW ---> LAW : 3
PENALTY ---> MONEY : 3
EVENT ---> STATE_OR_PROVINCE : 3
DISEASE ---> DISEASE : 3
PROFESSION ---> LOCATION : 2
LAW ---> COUNTRY : 2
PROFESSION ---> CITY : 2
EVENT ---> WORK_OF_ART : 2
FACILITY ---> ORGANIZATION : 2
WORK_OF_ART ---> COUNTRY : 2
LAW ---> ORGANIZATION : 1
PRODUCT ---> ORGANIZATION : 1
CRIME ---> LAW : 1
PROFESSION ---> EVENT : 1
PROFESSION ---> PERSON : 1
ORGAN

In [471]:
#!g1.1
for e, text, (found, expected), rel_tag in zip(mis_tags, texts, found_expected, rel_tags):
    if e == ('PROFESSION', 'ORGANIZATION'):
        print(f'found    = {found}')
        print(f'expected = {expected}')
        print(f'{e[0]} ---> {e[1]} : {rel_tag}')
        print(text)

found    = 1
expected = 0
PROFESSION ---> ORGANIZATION : None
[CLS] руководителя Приволжского межрегионального координационного совета « Единой России » [SEP] Приволжского межрегионального координационного совета « Единой России » [SEP]                                                                              
found    = 1
expected = 0
PROFESSION ---> ORGANIZATION : None
[CLS] Вокалист британской группы The Prodigy [SEP] The Prodigy [SEP]                                                                                        
found    = 0
expected = 1
PROFESSION ---> ORGANIZATION : WORKPLACE
[CLS] глава комитета Совета Федерации по конституционному законодательству и госстроительству [SEP] комитета Совета Федерации по конституционному законодательству и госстроительству [SEP]                                                                        
found    = 0
expected = 1
PROFESSION ---> ORGANIZATION : WORKPLACE
[CLS] главарем клана Гамбино [SEP] Гамбино [SEP]                        

In [472]:
#!g1.1
for e, text, (found, expected), rel_tag in zip(mis_tags, texts, found_expected, rel_tags):
    if e == ('ORGANIZATION', 'COUNTRY'):
        print(f'found    = {found}')
        print(f'expected = {expected}')
        print(f'{e[0]} ---> {e[1]} : {rel_tag}')
        print(text)

found    = 0
expected = 1
ORGANIZATION ---> COUNTRY : HEADQUARTERED_IN
[CLS] 1 - го Украинского фронта СССР [SEP] СССР [SEP]                                                                                          
found    = 1
expected = 0
ORGANIZATION ---> COUNTRY : None
[CLS] Приволжского межрегионального координационного совета « Единой России » [SEP] России [SEP]                                                                                       
found    = 0
expected = 1
ORGANIZATION ---> COUNTRY : HEADQUARTERED_IN
[CLS] Российской Федерации Северной ходьбы [SEP] Российской [SEP]                                                                                            
found    = 1
expected = 0
ORGANIZATION ---> COUNTRY : None
[CLS] Суд в Италии [SEP] Италии [SEP]                                                                                             
found    = 0
expected = 1
ORGANIZATION ---> COUNTRY : HEADQUARTERED_IN
[CLS] Итальянская прокуратура города Павия [SEP] Ита

In [473]:
#!g1.1
for e, text, (found, expected), rel_tag in zip(mis_tags, texts, found_expected, rel_tags):
    if e == ('EVENT', 'LOCATION'):
        print(f'found    = {found}')
        print(f'expected = {expected}')
        print(f'{e[0]} ---> {e[1]} : {rel_tag}')
        print(text)

found    = 0
expected = 1
EVENT ---> LOCATION : TAKES_PLACE_IN
[CLS] встречи на Эльбе [SEP] Эльбе [SEP]                                                                                             
found    = 0
expected = 1
EVENT ---> LOCATION : TAKES_PLACE_IN
[CLS] Встреча на Эльбе [SEP] Эльбе [SEP]                                                                                             
found    = 0
expected = 1
EVENT ---> LOCATION : TAKES_PLACE_IN
[CLS] Встреча на Эльбе [SEP] Эльбе [SEP]                                                                                             
found    = 0
expected = 1
EVENT ---> LOCATION : TAKES_PLACE_IN
[CLS] встречи на Эльбе [SEP] Эльбе [SEP]                                                                                             
found    = 0
expected = 1
EVENT ---> LOCATION : TAKES_PLACE_IN
[CLS] встречи на Эльбе [SEP] Эльбе [SEP]                                                                                             
found    = 0
ex

In [474]:
#!g1.1
for e, text, (found, expected), rel_tag in zip(mis_tags, texts, found_expected, rel_tags):
    if e == ('EVENT', 'PERSON'):
        print(f'found    = {found}')
        print(f'expected = {expected}')
        print(f'{e[0]} ---> {e[1]} : {rel_tag}')
        print(text)

found    = 0
expected = 1
EVENT ---> PERSON : PARTICIPANT_IN
[CLS] Прямой линии с Владимиром Путиным [SEP] Владимиром Путиным [SEP]                                                                                          
found    = 0
expected = 1
EVENT ---> PERSON : ORIGINS_FROM
[CLS] Международного конкурса имени Петра Чайковского [SEP] Петра Чайковского [SEP]                                                                                          
found    = 0
expected = 1
EVENT ---> PERSON : ORIGINS_FROM
[CLS] Конкурса имени Чайковского [SEP] Чайковского [SEP]                                                                                            
found    = 1
expected = 0
EVENT ---> PERSON : None
[CLS] Марше Голунова [SEP] Голунова [SEP]                                                                                           
found    = 0
expected = 1
EVENT ---> PERSON : ORIGINS_FROM
[CLS] Шукшинским дням на Алтае [SEP] Шукшинским [SEP]                                         